In [ ]:
from wiki_corpus_parser import WikiCorpus
print("Categories: ", ", ".join(WikiCorpus().categories))
datasetLoader = WikiCorpus(split_level="sen", category="ALL")
datasetLoader.create_csv()

In [ ]:
from jest_parser import JEST
JEST().create_csv()